# **SD202 - LAB 2 - QUENIAT HUGO**

# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
BUVEUR:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOCATION:
  0: LIEU(TEXT)
  1: REGION(TEXT)
M1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
M2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCER:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
WINE1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: MILL(NUM)
  3: DEGRE(NUM)
WINE2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: MILL(NUM)
  3: DEGRE(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [5]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [6]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)

Content of dummy
 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [7]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;


 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [8]:
# Write corresponding code here


%sql CREATE TABLE M1 AS \
SELECT DISTINCT NP, NV, QTE \
    FROM MASTER1;
    
%sql CREATE TABLE WINE1 AS \
SELECT DISTINCT NV, CRU, MILL, DEGRE \
    FROM MASTER1 \

%sql CREATE TABLE PRODUCER AS \
SELECT DISTINCT NP, NOM, PRENOM, REGION \
    FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of M1")
%sql SELECT * FROM M1

print("\nContent of WINE1")
%sql SELECT * FROM WINE1

print("\nContent of PRODUCER")
%sql SELECT * FROM PRODUCER

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
M1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCER:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
WINE1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: MILL(NUM)
  3: DEGRE(NUM)

Content of M1
 * sqlite:///wine.db
Done.

Content of WINE1
 * sqlite:///wine.db
Done.

Content of PRODUCER
 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


__1.4__ Create the new tables from Master2:

In [9]:
# Write corresponding code here

%sql CREATE TABLE M2 AS \
SELECT DISTINCT NV, NB, DATES, LIEU, QTE \
    FROM MASTER2;

%sql CREATE TABLE BUVEUR AS \
SELECT DISTINCT NB, NOM, PRENOM, TYPE \
    FROM MASTER2;
    
%sql CREATE TABLE LOCATION AS \
SELECT DISTINCT LIEU, REGION \
    FROM MASTER2;

%sql CREATE TABLE WINE2 AS \
SELECT DISTINCT NV, CRU, MILL, DEGRE \
    FROM MASTER2;
    
print("\nContent of the database")
printSchema(conn)

print("\nContent of M2")
%sql SELECT * FROM M2

print("\nContent of BUVEUR")
%sql SELECT * FROM BUVEUR

print("\nContent of LOCATION")
%sql SELECT * FROM LOCATION

print("\nContent of WINE2")
%sql SELECT * FROM WINE2
    

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
BUVEUR:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOCATION:
  0: LIEU(TEXT)
  1: REGION(TEXT)
M1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
M2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCER:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
WINE1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: MILL(NUM)
  3: DEGRE(NUM)
WINE2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: MILL(NUM)
  3: DEGRE(NUM)

Content of M2
 * sqlite:///wine.db
Done.

Content of BUVEUR
 * sqlite:///wine.db
Don

NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [10]:
%sql SELECT DISTINCT TYPE FROM BUVEUR WHERE TYPE IS NOT NULL;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


__2.2__ What regions produce Pommard or Brouilly?

In [11]:
%sql SELECT DISTINCT W.CRU, P.REGION FROM PRODUCER as P JOIN M1 on P.NP = M1.NP JOIN WINE1 as W  on W.NV = M1.NV WHERE  W.CRU = 'Pommard' or  W.CRU = 'Brouilly'; 

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


__2.3__ What regions produce Pommard and Brouilly?

In [12]:
%sql SELECT DISTINCT P.REGION FROM PRODUCER as P JOIN M1 on P.NP = M1.NP JOIN WINE1 as W  on W.NV = M1.NV WHERE  W.CRU = 'Pommard' INTERSECT SELECT DISTINCT P.REGION FROM PRODUCER as P JOIN M1 on P.NP = M1.NP JOIN WINE1 as W  on W.NV = M1.NV WHERE  W.CRU = 'Brouilly';

 * sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bught by CRU and Millésime

In [14]:
%sql SELECT W.CRU, W.MILL, SUM(M2.QTE) AS Number_of_wines_bought FROM BUVEUR as B JOIN M2 on B.NB = M2.NB JOIN WINE2 as W on W.NV = M2.NV GROUP BY W.CRU, W.MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [15]:
%sql SELECT NV, Number_of_producers FROM (SELECT W.NV, COUNT(P.NP) AS Number_of_producers FROM WINE1 as W JOIN MASTER1 AS M1 on W.NV = M1.NV JOIN PRODUCER as P on P.NP = M1.NP GROUP BY W.NV) WHERE Number_of_producers > 3;


 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

In [17]:
%sql SELECT NP, NOM, PRENOM FROM PRODUCER NATURAL JOIN M1 WHERE QTE =0 or QTE is NULL;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

In [18]:
%sql SELECT DISTINCT B.NB, B.NOM, B.PRENOM FROM BUVEUR as B JOIN M2 on B.NB = M2.NB JOIN WINE2 as W on W.NV = M2.NV WHERE W.MILL = 1980 and (M2.QTE is not NULL and M2.QTE > 0);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
44,Gide,Andre
45,Giono,Jean
8,Aragon,Louis
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

In [21]:
%sql SELECT DISTINCT B.NB, B.NOM, B.PRENOM FROM BUVEUR as B JOIN M2 on B.NB = M2.NB  EXCEPT SELECT DISTINCT B.NB, B.NOM, B.PRENOM FROM BUVEUR as B JOIN M2 on B.NB = M2.NB JOIN WINE2 as W on W.NV = M2.NV WHERE W.MILL = 1980 and M2.QTE > 0;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


__2.9__ What clients (buveurs) have bought ONLY wines from 1980?

In [22]:
%sql SELECT DISTINCT B.NB, B.NOM, B.PRENOM FROM BUVEUR as B JOIN M2 on B.NB = M2.NB JOIN WINE2 as W on W.NV = M2.NV WHERE W.MILL=1980 and M2.QTE > 0 EXCEPT SELECT DISTINCT B.NB, B.NOM, B.PRENOM FROM BUVEUR as B JOIN M2 on B.NB = M2.NB JOIN WINE2 as W on W.NV = M2.NV WHERE W.MILL<>1980 and M2.QTE > 0;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


__2.10__ List all wines from 1980

In [23]:
%sql SELECT NV, CRU, MILL, DEGRE FROM WINE1 WHERE MILL = 1980 UNION SELECT NV, CRU, MILL, DEGRE FROM WINE2 WHERE MILL = 1980;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


__2.11__ What are the wines from 1980 bought by NB=2?

In [24]:
%sql SELECT NV, NB, DATES, LIEU, QTE, CRU, MILL, DEGRE FROM M2 NATURAL JOIN (SELECT NV, CRU, MILL, DEGRE FROM WINE2 WHERE MILL = 1980) WHERE NB = 2;

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE,CRU,MILL,DEGRE
1,2,1977-11-02,BORDEAUX,33,Mercurey,1980,11.5


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

In [25]:
%sql SELECT B.NB, B.NOM, B.PRENOM FROM BUVEUR AS B NATURAL JOIN M2 NATURAL JOIN WINE2 AS W WHERE W.MILL = 1980 GROUP BY B.NB HAVING COUNT(distinct W.NV) = (SELECT COUNT(NV) FROM WINE2 WHERE MILL = 1980);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
